**Importing Libraries**

In [1]:
import logging
import os
import time
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**Data Ingestion**

In [2]:
# Get the current working directory
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

The current working directory is: C:\Users\nisha\Data Analysis Projects\ETL_Project


In [3]:
logging.basicConfig(
    filename = "logs/sql_ingestion_db_v2.log",
    level = logging.DEBUG, 
    format = "%(asctime)s - %(levelname)s - %(message)s",
    filemode = "a")

engine = create_engine('sqlite:///database.db')

def load_data(): 
    start = time.time()
    for file in os.listdir('data'): 
        if ".csv" in file: 
            df = pd.read_csv('data/'+ file) 
            print(file, " file_shape: " ,df.shape)
            logging.info(f'Ingesting {file} in db')
            df.to_sql(file[:-4], engine, if_exists='replace', index = False) 
    end = time.time()
    total_time = (end - start)/60
    logging.info('--------------Ingestion Complete------------')
    logging.info(f'\nTotal Time Taken: {total_time} minutes')

if __name__ == '__main__':
    load_data()

begin_inventory.csv  file_shape:  (8000, 9)
end_inventory.csv  file_shape:  (8000, 9)
purchases.csv  file_shape:  (32300, 16)
purchase_prices.csv  file_shape:  (12000, 9)
sales.csv  file_shape:  (95050, 14)
vendor_invoice.csv  file_shape:  (5200, 10)


**Data Loading Verification**

In [4]:
tables = pd.read_sql_query("SELECT name, type FROM sqlite_master WHERE type = 'table' LIMIT 10", engine)
tables

,name,type
0,begin_inventory,table
1,end_inventory,table
2,purchases,table
3,purchase_prices,table
4,sales,table
5,vendor_invoice,table


**Data Aggregation + Data Cleaning**

In [5]:
for table in tables['name']:   # 'name' should be in inverted commas as we are referring to a dataframe column
    print('Table Name:', table, '\nCount of records:', pd.read_sql_query(f" SELECT count(*) as count FROM {table}", engine)['count'].iloc[0])

Table Name: begin_inventory 
Count of records: 8000
Table Name: end_inventory 
Count of records: 8000
Table Name: purchases 
Count of records: 32300
Table Name: purchase_prices 
Count of records: 12000
Table Name: sales 
Count of records: 95050
Table Name: vendor_invoice 
Count of records: 5200


**Displaying Tables**

In [6]:
for table in tables['name']:   # 'name' should be in inverted commas as we are referring to a dictionary (sqlite_master) key
    print('-'*50, table, '-'*50)
    print("Count of Records: ", pd.read_sql_query(f"SELECT count(*) as count FROM {table}", engine)['count'].iloc[0])
    display(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5", engine))   # {table} in f-string uses the value stored in the table variable, using only 'table' will give an error

-------------------------------------------------- begin_inventory --------------------------------------------------
Count of Records:  8000


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,100000,Store_8,Toddfurt,Brand_21,Product_71,Medium,343,82.44,"Jun 24, 2024"
1,100001,Store_9,Port Shawn,Brand_3,Product_87,Small,241,157.82,2024-01-17
2,100002,Store_9,Lisaside,Brand_2,Product_112,Small,544,251.95,05/09/2024
3,100003,Store_5,South Danielle,Brand_12,Product_31,Small,323,156.20,24/08/24
4,100004,Store_8,Toddfurt,Brand_24,Product_179,Large,31,193.32,06/06/2024


-------------------------------------------------- end_inventory --------------------------------------------------
Count of Records:  8000


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,200000,Store_4,Lake Davidchester,Brand_19,Product_124,Medium,159,173.20,"Aug 16, 2024"
1,200001,Store_13,New Dawn,Brand_6,Product_140,Large,588,232.66,10/11/2024
2,200002,Store_9,Loriland,Brand_22,Product_67,Large,556,113.63,"Feb 23, 2024"
3,200003,Store_15,South Danielle,Brand_25,Product_52,Large,360,241.09,"Jul 17, 2024"
4,200004,Store_9,Shanemouth,Brand_17,Product_23,Small,642,285.39,2024-01-04


-------------------------------------------------- purchases --------------------------------------------------
Count of Records:  32300


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,Brand_7,Product_143,111.24,Medium,6,A,125.27,V018,Long-Roberts
1,Brand_20,Product_169,226.35,Large,12,C,223.92,V013,Jackson Group
2,Brand_15,Product_115,252.93,Small,12,C,174.18,V031,Harris PLC
3,Brand_11,Product_109,255.70,Small,6,B,222.85,V003,"Pratt, Parrish and Reilly"
4,Brand_8,Product_90,254.02,Medium,24,C,242.18,V030,Banks-Montgomery


-------------------------------------------------- sales --------------------------------------------------
Count of Records:  95050


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesPrice,SalesDate,Volume,Classification,VendorNo,VendorName,SalesDollars,ExciseTax
0,400000,Store_11,brand_9,Product_94,Large,352.0,211.36,06/09/2024,5292,C,V004,Keith-Sanchez,74398.72,3943.99
1,400001,Store_14,brand_12,Product_29,Small,58.0,244.67,03/10/2024,8851,B,V017,Harrington-Gray,14190.86,837.90
2,400002,Store_3,Brand_21,Product_119,Medium,57.0,200.76,"Dec 15, 2024",7843,C,V031,Kane_and_Sons,11443.32,1437.55
3,400003,Store_1,Brand 25,Product_163,Large,354.0,228.12,"Jan 11, 2024",1376,C,V026,"FUENTES, RICHARDS AND JOHNSON",80754.48,11208.05
4,400004,Store_16,BRD-3,Product_18,Medium,334.0,221.32,"May 29, 2024",7600,C,V005,cordova llc,73920.88,9542.55


-------------------------------------------------- vendor_invoice --------------------------------------------------
Count of Records:  5200


,VendorNumber,VendorName,Quantity,Dollars,Freight,InvoiceDate,PODate,PayDate,PONumber,Approval
0,V039,Lara_Group,212337.0,36910458.55,273137.39,"Apr 08, 2024",07/26/2024,2024-10-28,PO-8697,Pending
1,V029,Mcdaniel-Le,199724.0,34347016.44,501466.44,2024-05-30,03/21/2024,09/07/2024,PO-1759,Approved
2,V015,Wolfe-Murillo Ltd,217758.0,37421596.92,508933.72,30/04/24,"Jan 30, 2024","Dec 09, 2024",PO-7385,Approved
3,V008,MCDONALD LTD,207022.0,35699470.20,560481.68,"Mar 28, 2024","Jan 30, 2024","Feb 04, 2024",PO-0551,Approved
4,V021,BANKS-MONTGOMERY,199468.0,34367702.77,457090.45,28/12/24,"Oct 31, 2024",17/09/24,PO-5749,Approved


**Data Cleaning -- Date Formatting**

In [7]:
#LOADING TABLES INTO PYTHON MEMORY AS DATAFRAMES - Python has rihcer datetime parsing/ formatting functions

#Approach 1: for table in tables['name']: name = pd.read_sql_query(f"SELECT * FROM {table}", engine) -- But this is helpful when the operations are being performed WHILE running the loop itself

#Approach 2:
dfs = {df: pd.read_sql_query(f"SELECT * FROM {df}", engine) for df in tables['name']}

In [8]:
#CONVERTING TO DATETIME FORMAT 

#dfs['begin_inventory']['startDate'].apply(type).value_counts()  # chaining keys of dictionary dfs to reference each table

dfs['begin_inventory']['startDate_clean'] = pd.to_datetime(dfs['begin_inventory']['startDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['end_inventory']['endDate_clean'] = pd.to_datetime(dfs['end_inventory']['endDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['purchases']['ReceivingDate_clean'] = pd.to_datetime(dfs['purchases']['ReceivingDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['purchases']['InvoiceDate_clean'] = pd.to_datetime(dfs['purchases']['InvoiceDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['purchases']['PayDate_clean'] = pd.to_datetime(dfs['purchases']['PayDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['vendor_invoice']['PayDate_clean'] = pd.to_datetime(dfs['purchases']['PayDate'],format='mixed', errors='coerce',dayfirst=True)
dfs['vendor_invoice']['PODate_clean'] = pd.to_datetime(dfs['purchases']['PODate'],format='mixed', errors='coerce',dayfirst=True)

**Data Cleaning - Standardizing Text Columns**

In [9]:
## REMOVING TRAILING SPACES FROM ALL TEXT COLUMNS

for name, df in dfs.items():
    for col_name in df.columns:
        if df[col_name].dtype == 'object':
            df[col_name] = df[col_name].astype(str).str.strip()

# Other Approach using isinstance() method: 
#for name, df in dfs.items():
#    for col_name in df.columns:    
#        if df[col_name].apply(lambda x: isinstance(x, str)).all():
#            df[col_name].str.strip()

#-----------------------------------------------------------------

In [10]:
## CHECKING THE ENTRIES IN THE TEXT COLUMNS TO IDENTIFY INCONSISTENCIES

for name, df in dfs.items():
    for col_name in df.columns:
        if  df[col_name].dtype == 'object':
            print(f"-----{name}-----:----{col_name}----")
            print(df[col_name].sort_values().unique(),"\n")

## ONLY WORKING WITH COLUMNS THAT ARE IMPORTANT FOR ANALYSIS
for name, df in dfs.items():
    for col_name in df.columns:
        if (df[col_name].dtype == 'object') & (col_name in ['City','Brand','VendorName']):
            print(f"-----{name}-----:----{col_name}----")
            print(df[col_name].sort_values().unique(),"\n")

#----------------------------------------------------------------

-----begin_inventory-----:----Store----
['Store_1' 'Store_10' 'Store_11' 'Store_12' 'Store_13' 'Store_14'
 'Store_15' 'Store_16' 'Store_17' 'Store_18' 'Store_2' 'Store_3' 'Store_4'
 'Store_5' 'Store_6' 'Store_7' 'Store_8' 'Store_9'] 

-----begin_inventory-----:----City----
['Adambury' 'Crossshire' 'East Jessicastad' 'Lake Davidchester'
 'Lake Ryanmouth' 'Lisaside' 'Loriland' 'New Dawn' 'Port Brett'
 'Port Joshua' 'Port Shawn' 'Shanemouth' 'South Danielle' 'South Peter'
 'South Victorberg' 'Stevenmouth' 'Toddfurt' 'West Amber'] 

-----begin_inventory-----:----Brand----
['Brand_1' 'Brand_10' 'Brand_11' 'Brand_12' 'Brand_13' 'Brand_14'
 'Brand_15' 'Brand_16' 'Brand_17' 'Brand_18' 'Brand_19' 'Brand_2'
 'Brand_20' 'Brand_21' 'Brand_22' 'Brand_23' 'Brand_24' 'Brand_25'
 'Brand_3' 'Brand_4' 'Brand_5' 'Brand_6' 'Brand_7' 'Brand_8' 'Brand_9'
 'brand_1' 'brand_10' 'brand_11' 'brand_12' 'brand_13' 'brand_14'
 'brand_15' 'brand_16' 'brand_18' 'brand_19' 'brand_2' 'brand_21'
 'brand_22' 'brand_23' 

In [11]:
## DEFINING A FUNCTION TO RESOLVE ALL INCONSISTENCIES AT ONCE

def clean_brand(x):
    if pd.isna(x): 
        return None
    x = x.lower()                     
    x = x.replace('-','_')
    x = x.replace('BRD-','Brand_')
    x = x.replace('brd','Brand')
    x = x.replace('Brand ', 'Brand_') # converts Brand 1 → Brand_1
    x = x.replace('-', '_')           # handles any leftover dashes   
    x = x.title()                     # makes “brand_1” → “Brand_1”
    return x


## APPLYING THE FUNCTIONS TO THE REQUIRED TABLES
for name in ['begin_inventory', 'end_inventory', 'purchases', 'purchase_prices', 'sales']:
    dfs[name]['Brand_clean'] = dfs[name]['Brand'].apply(clean_brand)

#----------------------------------------------------------------

In [12]:
## DEFINING A FUNCTION TO RESOLVE ALL INCONSISTENCIES AT ONCE

def clean_vendor_name(x):
    if pd.isna(x):
        return None
    x = x.lower()
    x = x.replace('-', ' ').replace('_', ' ').replace('.', ' ')
    x = x.replace('ltd ','Ltd')
    x = x.replace('ltd','Ltd')
    x = x.replace('limited ','Ltd')
    x = x.replace('Inc ','Inc')
    x = x.replace('llc','LLC')
    x = x.title()
    return x

## APPLYING THE FUNCTIONS TO THE REQUIRED TABLES
for name in ['purchases', 'purchase_prices', 'sales', 'vendor_invoice']:
    dfs[name]['VendorName_clean'] = dfs[name]['VendorName'].apply(clean_vendor_name)

#----------------------------------------------------------------

In [13]:
## CREATING A MASTER LIST OF ALL VALUES FOR FINAL REVIEW

master_vendor_names = pd.concat([
    dfs['purchases']['VendorName_clean'],
    dfs['purchase_prices']['VendorName_clean'],
    dfs['sales']['VendorName_clean'],
    dfs['vendor_invoice']['VendorName_clean']]).dropna().unique()

pd.Series(master_vendor_names).sort_values()

## MANUALLY MAPPING THE CORRECT NAMES

vendor_names_map = {
    'Estradamendoza': 'Estradamendoza Ltd', 
    'Fitzgeraldllc':'Fitzgerald Llc',
    'Garza Llc':'Garza Llc Ltd',
    'Moranbray': 'Moranbray Ltd',
    'Nguyen Group': 'Nguyen Group Ltd', 
    'Nguyenthomas': 'Nguyenthomas Ltd', 
    'Snyder, Scott And Haas': 'Snyder, Scott And Haas Ltd', 
    'Thompson Group': 'Thompson Group', 
    'Williams,Hickmanandnichols':'Williams, Hickman And Nichols'}

for name in ['purchases', 'purchase_prices', 'sales', 'vendor_invoice']:
    dfs[name]['VendorName_clean'] = dfs[name]['VendorName_clean'].replace(vendor_names_map)

In [14]:
for name, df in dfs.items():  
    print('-'*50, name, '-'*50)
    display(df)   

-------------------------------------------------- begin_inventory --------------------------------------------------


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate,startDate_clean,Brand_clean
0,100000,Store_8,Toddfurt,Brand_21,Product_71,Medium,343,82.44,"Jun 24, 2024",2024-06-24,Brand_21
1,100001,Store_9,Port Shawn,Brand_3,Product_87,Small,241,157.82,2024-01-17,2024-01-17,Brand_3
2,100002,Store_9,Lisaside,Brand_2,Product_112,Small,544,251.95,05/09/2024,2024-09-05,Brand_2
3,100003,Store_5,South Danielle,Brand_12,Product_31,Small,323,156.20,24/08/24,2024-08-24,Brand_12
4,100004,Store_8,Toddfurt,Brand_24,Product_179,Large,31,193.32,06/06/2024,2024-06-06,Brand_24
...,...,...,...,...,...,...,...,...,...,...,...
7995,107995,Store_9,East Jessicastad,Brand_4,Product_117,Medium,738,136.49,12/08/24,2024-08-12,Brand_4
7996,107996,Store_13,South Danielle,Brand_6,Product_58,Small,10,299.58,"Aug 09, 2024",2024-08-09,Brand_6
7997,107997,Store_18,South Peter,Brand_16,Product_155,Medium,676,89.28,"Nov 08, 2024",2024-11-08,Brand_16
7998,107998,Store_14,South Peter,Brand_19,Product_64,Medium,553,167.78,"Apr 24, 2024",2024-04-24,Brand_19


-------------------------------------------------- end_inventory --------------------------------------------------


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate,endDate_clean,Brand_clean
0,200000,Store_4,Lake Davidchester,Brand_19,Product_124,Medium,159,173.20,"Aug 16, 2024",2024-08-16,Brand_19
1,200001,Store_13,New Dawn,Brand_6,Product_140,Large,588,232.66,10/11/2024,2024-11-10,Brand_6
2,200002,Store_9,Loriland,Brand_22,Product_67,Large,556,113.63,"Feb 23, 2024",2024-02-23,Brand_22
3,200003,Store_15,South Danielle,Brand_25,Product_52,Large,360,241.09,"Jul 17, 2024",2024-07-17,Brand_25
4,200004,Store_9,Shanemouth,Brand_17,Product_23,Small,642,285.39,2024-01-04,2024-01-04,Brand_17
...,...,...,...,...,...,...,...,...,...,...,...
7995,207995,Store_16,South Danielle,Brand_9,Product_8,Small,458,135.46,15/06/24,2024-06-15,Brand_9
7996,207996,Store_17,New Dawn,Brand_23,Product_25,Small,10,98.20,2024-09-09,2024-09-09,Brand_23
7997,207997,Store_3,Toddfurt,Brand_14,Product_2,Large,816,194.73,2024-01-27,2024-01-27,Brand_14
7998,207998,Store_13,South Peter,Brand_3,Product_110,Small,316,327.70,09/01/2024,2024-01-09,Brand_3


-------------------------------------------------- purchases --------------------------------------------------


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Brand_clean,VendorName_clean
0,Brand_7,Product_143,111.24,Medium,6,A,125.27,V018,Long-Roberts,Brand_7,Long Roberts
1,Brand_20,Product_169,226.35,Large,12,C,223.92,V013,Jackson Group,Brand_20,Jackson Group
2,Brand_15,Product_115,252.93,Small,12,C,174.18,V031,Harris PLC,Brand_15,Harris Plc
3,Brand_11,Product_109,255.70,Small,6,B,222.85,V003,"Pratt, Parrish and Reilly",Brand_11,"Pratt, Parrish And Reilly"
4,Brand_8,Product_90,254.02,Medium,24,C,242.18,V030,Banks-Montgomery,Brand_8,Banks Montgomery
...,...,...,...,...,...,...,...,...,...,...,...
11995,Brand_25,Product_91,298.41,Small,6,A,130.93,V005,Parker-Rogers,Brand_25,Parker Rogers
11996,Brand_19,Product_132,117.69,Large,12,C,208.41,V027,"Reese, Baker and Sutton",Brand_19,"Reese, Baker And Sutton"
11997,Brand_3,Product_2,203.85,Small,24,B,105.83,V012,Gaines-Mckinney,Brand_3,Gaines Mckinney
11998,Brand_4,Product_115,159.13,Small,12,B,278.30,V010,"Reese, Baker and Sutton",Brand_4,"Reese, Baker And Sutton"


-------------------------------------------------- sales --------------------------------------------------


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesPrice,SalesDate,Volume,Classification,VendorNo,VendorName,SalesDollars,ExciseTax,Brand_clean,VendorName_clean
0,400000,Store_11,brand_9,Product_94,Large,352.0,211.36,06/09/2024,5292,C,V004,Keith-Sanchez,74398.72,3943.99,Brand_9,Keith Sanchez
1,400001,Store_14,brand_12,Product_29,Small,58.0,244.67,03/10/2024,8851,B,V017,Harrington-Gray,14190.86,837.90,Brand_12,Harrington Gray
2,400002,Store_3,Brand_21,Product_119,Medium,57.0,200.76,"Dec 15, 2024",7843,C,V031,Kane_and_Sons,11443.32,1437.55,Brand_21,Kane And Sons
3,400003,Store_1,Brand 25,Product_163,Large,354.0,228.12,"Jan 11, 2024",1376,C,V026,"FUENTES, RICHARDS AND JOHNSON",80754.48,11208.05,Brand 25,"Fuentes, Richards And Johnson"
4,400004,Store_16,BRD-3,Product_18,Medium,334.0,221.32,"May 29, 2024",7600,C,V005,cordova llc,73920.88,9542.55,Brand_3,Cordova Llc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95045,464255,Store_16,brand_14,Product_41,Large,296.0,194.11,08/28/2024,5035,A,V009,Jackson Group,57456.56,8436.10,Brand_14,Jackson Group
95046,404341,Store_10,BRD-6,Product_40,Medium,93.0,219.55,"Jul 27, 2024",10174,C,V012,Hill LLC,20418.15,1554.78,Brand_6,Hill Llc
95047,481145,Store_1,BRD-7,Product_65,Medium,80.0,212.76,10/05/24,7168,B,V036,Mclaughlin-Smith,17020.80,1313.53,Brand_7,Mclaughlin Smith
95048,492573,Store_18,Brand 17,Product_34,Medium,168.0,185.08,06/13/2024,9745,C,V022,Gaines-Mckinney Ltd,31093.44,4168.63,Brand 17,Gaines Mckinney Ltd


-------------------------------------------------- vendor_invoice --------------------------------------------------


,VendorNumber,VendorName,Quantity,Dollars,Freight,InvoiceDate,PODate,PayDate,PONumber,Approval,PayDate_clean,PODate_clean,VendorName_clean
0,V039,Lara_Group,212337.0,36910458.55,273137.39,"Apr 08, 2024",07/26/2024,2024-10-28,PO-8697,Pending,2024-07-17,2024-03-04,Lara Group
1,V029,Mcdaniel-Le,199724.0,34347016.44,501466.44,2024-05-30,03/21/2024,09/07/2024,PO-1759,Approved,2024-11-24,2024-06-09,Mcdaniel Le
2,V015,Wolfe-Murillo Ltd,217758.0,37421596.92,508933.72,30/04/24,"Jan 30, 2024","Dec 09, 2024",PO-7385,Approved,2024-05-26,2024-05-14,Wolfe Murillo Ltd
3,V008,MCDONALD LTD,207022.0,35699470.20,560481.68,"Mar 28, 2024","Jan 30, 2024","Feb 04, 2024",PO-0551,Approved,2024-07-23,2024-04-20,Mcdonald Ltd
4,V021,BANKS-MONTGOMERY,199468.0,34367702.77,457090.45,28/12/24,"Oct 31, 2024",17/09/24,PO-5749,Approved,2024-01-18,2024-07-25,Banks Montgomery
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,V001,"Pratt, Parrish and Reilly",202883.0,34883264.26,216276.24,06/01/2024,2024-03-28,2024-02-18,PO-1810,Pending,2024-06-02,2024-10-24,"Pratt, Parrish And Reilly"
5196,V001,"Pratt, Parrish and Reilly",202883.0,34883264.26,216276.24,06/01/2024,2024-03-28,2024-02-18,PO-1810,Pending,2024-08-18,2024-07-20,"Pratt, Parrish And Reilly"
5197,V017,Harrington-Gray,214295.0,36857451.30,353831.53,07/19/2024,23/09/24,10/12/24,PO-5843,Approved,2024-06-28,2024-11-23,Harrington Gray
5198,None,Long-Roberts,207956.0,35895805.88,341010.16,"Feb 02, 2024",11/17/2024,2024-04-01,PO-5662,Approved,2024-05-24,2024-11-17,Long Roberts


**Data Cleaning -- Duplicates**

In [15]:
for name, df in dfs.items():
    df.drop_duplicates(keep='first')
    print(df.shape)

(8000, 11)
(8000, 11)
(32300, 21)
(12000, 11)
(95050, 16)
(5200, 13)


**Data Cleaning -- Missing Values**

In [16]:
## CHECKING NON-NULL COUNTS ACROSS ALL TABLES

#Approach 1: Recommended
for name, df in dfs.items():
    print(f"\n--- {name} ---")
    df.info()  # .info() is a method and needs to be called on the dataframe

#Approach 2: Ad-hoc for the required table
#for col_name in dfs['purchases'].columns: 
#    null_count = dfs['purchases'][col_name].isna().sum()
#    print(f"{col_name}: {null_count}") ##


--- begin_inventory ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   InventoryId      8000 non-null   int64         
 1   Store            8000 non-null   object        
 2   City             8000 non-null   object        
 3   Brand            8000 non-null   object        
 4   Description      8000 non-null   object        
 5   Size             8000 non-null   object        
 6   onHand           8000 non-null   int64         
 7   Price            8000 non-null   float64       
 8   startDate        8000 non-null   object        
 9   startDate_clean  8000 non-null   datetime64[ns]
 10  Brand_clean      8000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 687.6+ KB

--- end_inventory ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999

In [17]:
# FILLING MISSING VALUES WITH '0' AS <1% 

dfs['vendor_invoice']['VendorNumber'] = dfs['vendor_invoice']['VendorNumber'].astype(str).str.replace("None","Unknown")
dfs['begin_inventory']['Description'] = dfs['begin_inventory']['Description'].astype(str).str.replace("None","Unknown")
dfs['sales'].fillna(0,inplace = True)
dfs['purchases'].fillna(0,inplace = True)

**Misc -- Data Checks**

In [19]:
#checking cnnecting link between tables
pd.read_sql_query("SELECT * FROM purchase_prices AS p JOIN vendor_invoice as v ON CAST(p.VendorNumber AS TEXT) = CAST(v.VendorNumber AS TEXT)", engine)['VendorNumber'].nunique()

VendorNumber    40
VendorNumber    40
dtype: int64

In [15]:
# Step 1: Check how many IDs overlap
set_b = set(pd.read_sql("SELECT InventoryId FROM begin_inventory", engine)["InventoryId"])
set_e = set(pd.read_sql("SELECT InventoryId FROM end_inventory", engine)["InventoryId"])

print("Overlap %:", len(set_b & set_e) / len(set_b) * 100)

# Step 2: Check data types
print(pd.read_sql("PRAGMA table_info(begin_inventory)", engine))
print(pd.read_sql("PRAGMA table_info(end_inventory)", engine))

Overlap %: 0.0
   cid         name    type  notnull dflt_value  pk
0    0  InventoryId  BIGINT        0       None   0
1    1        Store    TEXT        0       None   0
2    2         City    TEXT        0       None   0
3    3        Brand    TEXT        0       None   0
4    4  Description    TEXT        0       None   0
5    5         Size    TEXT        0       None   0
6    6       onHand  BIGINT        0       None   0
7    7        Price   FLOAT        0       None   0
8    8    startDate    TEXT        0       None   0
   cid         name    type  notnull dflt_value  pk
0    0  InventoryId  BIGINT        0       None   0
1    1        Store    TEXT        0       None   0
2    2         City    TEXT        0       None   0
3    3        Brand    TEXT        0       None   0
4    4  Description    TEXT        0       None   0
5    5         Size    TEXT        0       None   0
6    6       onHand  BIGINT        0       None   0
7    7        Price   FLOAT        0       None  

In [17]:
# Step 1: Check how many Vendor IDs overlap
set_v = set(pd.read_sql("SELECT VendorNumber FROM vendor_invoice", engine)["VendorNumber"])
set_p = set(pd.read_sql("SELECT VendorNumber FROM purchase_prices", engine)["VendorNumber"])

print("Overlap %:", len(set_v & set_p) / len(set_p) * 100)

# Step 2: Check data types
print(pd.read_sql("PRAGMA table_info(purchase_prices)", engine))
print(pd.read_sql("PRAGMA table_info(vendor_invoice)", engine))

Overlap %: 97.5609756097561
   cid            name    type  notnull dflt_value  pk
0    0           Brand    TEXT        0       None   0
1    1     Description    TEXT        0       None   0
2    2           Price   FLOAT        0       None   0
3    3            Size    TEXT        0       None   0
4    4          Volume  BIGINT        0       None   0
5    5  Classification    TEXT        0       None   0
6    6   PurchasePrice   FLOAT        0       None   0
7    7    VendorNumber    TEXT        0       None   0
8    8      VendorName    TEXT        0       None   0
   cid          name    type  notnull dflt_value  pk
0    0  VendorNumber    TEXT        0       None   0
1    1    VendorName    TEXT        0       None   0
2    2   InvoiceDate    TEXT        0       None   0
3    3      PONumber    TEXT        0       None   0
4    4        PODate    TEXT        0       None   0
5    5       PayDate    TEXT        0       None   0
6    6      Quantity  BIGINT        0       None   

**Final Data - Post Cleaning**

In [18]:
for name, df in dfs.items():  
    print('-'*50, name, '-'*50)
    display(df)   

-------------------------------------------------- begin_inventory --------------------------------------------------


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate,startDate_clean,Brand_clean
0,100000,Store_7,Patrickhaven,Brand_16,Product_132,Medium,513,59.99,"Apr 07, 2024",2024-04-07,Brand_16
1,100001,Store_15,Kramerhaven,Brand_2,Product_66,Medium,135,182.61,03/03/24,2024-03-03,Brand_2
2,100002,Store_11,Smithside,Brand_15,Product_120,Small,628,150.98,14/01/24,2024-01-14,Brand_15
3,100003,Store_8,East Brittneyview,Brand_13,Product_124,Medium,408,156.96,"Mar 07, 2024",2024-03-07,Brand_13
4,100004,Store_7,Kramerhaven,Brand_13,Product_161,Large,120,83.44,2024-06-17,2024-06-17,Brand_13
...,...,...,...,...,...,...,...,...,...,...,...
7995,107995,Store_2,Port Vernon,Brand_23,Product_151,Medium,38,242.89,"Oct 03, 2024",2024-10-03,Brand_23
7996,107996,Store_6,Port Michael,Brand_23,Product_25,Medium,618,23.22,"Apr 08, 2024",2024-04-08,Brand_23
7997,107997,Store_18,Lake Emily,Brand_7,Product_37,Small,611,237.01,05/02/24,2024-02-05,Brand_7
7998,107998,Store_9,Dayhaven,Brand_21,Product_112,Small,783,48.22,"Jan 04, 2024",2024-01-04,Brand_21


-------------------------------------------------- end_inventory --------------------------------------------------


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate,endDate_clean,Brand_clean
0,200000,Store_12,East Sarah,Brand_12,Product_41,Medium,730,77.30,2024-08-22,2024-08-22,Brand_12
1,200001,Store_16,Port Vernon,Brand_15,Product_77,Large,75,98.68,02/09/24,2024-09-02,Brand_15
2,200002,Store_18,Smithside,Brand_18,Product_105,Small,201,252.74,04/13/2024,2024-04-13,Brand_18
3,200003,Store_5,Dayhaven,Brand_23,Product_49,Large,200,9.31,2024-06-08,2024-06-08,Brand_23
4,200004,Store_16,Port Daniellehaven,Brand_1,Product_71,Small,767,84.27,"Mar 01, 2024",2024-03-01,Brand_1
...,...,...,...,...,...,...,...,...,...,...,...
7995,207995,Store_15,Penamouth,Brand_10,Product_125,Large,541,222.64,26/04/24,2024-04-26,Brand_10
7996,207996,Store_16,Gibsonmouth,Brand_5,Product_180,Small,754,199.96,24/09/24,2024-09-24,Brand_5
7997,207997,Store_10,Rivasmouth,Brand_22,Product_169,Small,729,79.09,"Oct 15, 2024",2024-10-15,Brand_22
7998,207998,Store_4,Penamouth,Brand_11,Product_91,Small,323,97.83,2024-03-16,2024-03-16,Brand_11


-------------------------------------------------- purchases --------------------------------------------------


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Brand_clean,VendorName_clean
0,Brand_17,Product_105,31.90,Large,10640,A,162.36,V031,"Johnson, Jones and Martinez",Brand_17,"Johnson, Jones And Martinez"
1,Brand_16,Product_110,328.05,Small,4863,A,69.10,V025,Thomas Group,Brand_16,Thomas Group
2,Brand_13,Product_79,254.32,Large,14815,A,164.88,V030,"Wallace, Young and Rodriguez",Brand_13,"Wallace, Young And Rodriguez"
3,Brand_18,Product_70,74.81,Small,8468,A,265.02,V033,Gardner Inc,Brand_18,Gardner Inc
4,Brand_2,Product_153,77.17,Large,8798,C,249.02,V014,Anderson Ltd,Brand_2,Anderson Ltd
...,...,...,...,...,...,...,...,...,...,...,...
11995,Brand_9,Product_155,17.82,Small,13205,A,15.02,V007,Lewis-Davis,Brand_9,Lewis Davis
11996,Brand_10,Product_174,19.80,Large,12595,B,221.78,V022,"Johnson, Pace and Anderson",Brand_10,"Johnson, Pace And Anderson"
11997,Brand_5,Product_52,252.86,Small,9463,C,190.31,V028,Gardner Inc,Brand_5,Gardner Inc
11998,Brand_21,Product_141,66.94,Small,7204,B,95.71,V040,"Thomas, Morris and Murray",Brand_21,"Thomas, Morris And Murray"


-------------------------------------------------- sales --------------------------------------------------


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName,Brand_clean,VendorName_clean
0,400000,Store_5,Brand 16,Product_81,Small,468.0,53787.63,27.80,2024-06-27,2168,A,44.37,V028,Bowman and Sons,Brand 16,Bowman And Sons
1,400001,Store_3,brand_9,Product_23,Medium,254.0,73591.06,344.07,"Jun 06, 2024",4203,A,15.61,V037,"JOHNSON, JONES AND MARTINEZ",Brand_9,"Johnson, Jones And Martinez"
2,400002,Store_2,Brand 14,Product_149,Medium,422.0,51853.53,228.58,"Jul 26, 2024",7965,A,5.16,V012,Gardner Inc.,Brand 14,Gardner Inc
3,400003,Store_18,brand_20,Product_176,Medium,31.0,5273.03,385.70,13/04/24,6358,C,8.47,V025,Boyle-Prince Ltd,Brand_20,Boyle Prince Ltd
4,400004,Store_13,brand_15,Product_131,Medium,231.0,67543.52,212.74,2024-02-20,4296,C,59.72,V032,GREEN-WOODARD,Brand_15,Green Woodard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95020,479625,Store_4,Brand 13,Product_151,Small,455.0,791.10,95.63,2024-06-16,6652,C,52.22,V023,Smith and Sons Ltd,Brand 13,Smith And Sons Ltd
95021,411795,Store_4,BRD-16,Product_125,Small,144.0,6430.05,73.62,"May 04, 2024",3132,A,86.46,V020,ANDERSON LTD,Brand_16,Anderson Ltd
95022,485200,Store_11,Brand 3,Product_70,Medium,215.0,678.64,332.97,02/15/2024,11362,B,39.91,V026,"BALLARD, RAMIREZ AND CARTER",Brand 3,"Ballard, Ramirez And Carter"
95023,442775,Store_14,Brand_21,Product_179,Large,30.0,3676.29,176.13,2024-02-09,7242,C,86.00,V008,"JOHNSON, PACE AND ANDERSON",Brand_21,"Johnson, Pace And Anderson"


-------------------------------------------------- vendor_invoice --------------------------------------------------


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval,PayDate_clean,PODate_clean,VendorName_clean
0,V006,"Wallace, Young and Rodriguez",07/19/2024,PO-5191,"Sep 03, 2024",09/15/2024,758,28286.45,3103.04,Approved,2024-03-06,2024-03-24,"Wallace, Young And Rodriguez"
1,V014,"Yoder, Boyle and Williams",02/28/2024,PO-6825,11/04/2024,2024-12-14,715,83124.63,775.70,Approved,2024-09-07,2024-01-11,"Yoder, Boyle And Williams"
2,V025,"Yang,_Jones_and_Miller",2024-09-21,PO-0426,20/10/24,29/07/24,1498,14939.27,2087.73,Rejected,2024-05-31,2024-08-10,"Yang, Jones And Miller"
3,V015,"JOHNSON, PACE AND ANDERSON","Oct 10, 2024",PO-0826,11/26/2024,"Aug 18, 2024",838,76553.63,200.26,Pending,2024-01-23,2024-10-15,"Johnson, Pace And Anderson"
4,V007,ENGLISH-LAWSON,"Sep 17, 2024",PO-6597,23/10/24,07/23/2024,897,36160.29,4107.63,Approved,2024-11-10,2024-01-30,English Lawson
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,V015,Villanueva-Flores,05/10/24,PO-5978,11/08/2024,01/27/2024,1521,28697.01,4664.58,Approved,2024-07-10,2024-10-20,Villanueva Flores
5196,V013,"rodriguez, ferguson and murphy",20/02/24,PO-9678,12/08/2024,"Nov 18, 2024",1294,91874.68,3726.25,Approved,2024-02-10,2024-11-10,"Rodriguez, Ferguson And Murphy"
5197,V003,Parker Group Ltd,2024-09-15,PO-0983,11/27/2024,"Jul 02, 2024",1287,8420.97,2006.77,Approved,2024-07-08,2024-06-15,Parker Group Ltd
5198,V022,zimmerman llc,25/04/24,PO-8776,"Feb 18, 2024",13/01/24,1583,12125.88,2341.11,Approved,2024-10-22,2024-03-21,Zimmerman Llc


**Storing the Cleaned Tables back into CSVs**

In [18]:
output_folder = "cleaned_data"
os.makedirs(output_folder, exist_ok=True)

for name, df in dfs.items():
    output_path = f"{output_folder}/{name}.csv"  # output_path = cleaned_data/ table_name.csv
    df.to_csv(output_path, index=False)
    print(f"Saved: {output_path} ({df.shape[0]} rows, {df.shape[1]} columns)")

Saved: cleaned_data/begin_inventory.csv (8000 rows, 11 columns)
Saved: cleaned_data/end_inventory.csv (8000 rows, 11 columns)
Saved: cleaned_data/purchases.csv (32300 rows, 21 columns)
Saved: cleaned_data/purchase_prices.csv (12000 rows, 11 columns)
Saved: cleaned_data/sales.csv (95050 rows, 16 columns)
Saved: cleaned_data/vendor_invoice.csv (5200 rows, 13 columns)



**Understanding the Info in each table**

- The begin_inventory and end_inventory tables contain inventory data at the start and end of the year, which is not relevant for analyzing vendor behavior.
- Therefore, these tables can be excluded from the analysis.
- Next, we will examine how vendor-related data is distributed across other tables.

-------

- Purcahes: info about the procurements made by the Vendors from the company along with Trasnaction Details (Invoice Dates, Payment Dates, Transaction Value i.e. Dollars), Product (Brand) Prices ($), Quantity procured
- Purchase_Prices: The purchase price column is derived from the purchase_prices table, which provides product-wise actual and purchase prices. In addition to the Product, Brand, it provides info like Volume. 
- Vendor_Invoice: aggregates data from the purchases table, summarizing Quantity and Dollar amounts, along with an additional column for Freight
- Sales: contains the sales transactions for the year - Product Info, Brands purchased by vendors, Volume, SalesQuantity, SalesPrice, SalesDollars (revenue earned)

------

As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:
- purchase transactions made by vendors
- sales transaction data
- freight costs for each vendor
- actual product prices from vendors

In [80]:
wb1 = dfs['purchase_prices'][dfs['purchase_prices']['VendorNumber'] == 'V017'].sort_values('VendorName_clean')
wb1

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Brand_clean,VendorName_clean
6092,Brand_16,Product_47,267.45,Medium,10874,B,284.58,V017,"Anderson, Hernandez and Brown",Brand_16,"Anderson, Hernandez And Brown"
18931,Brand_15,Product_129,179.55,Medium,12650,B,231.74,V017,"Anderson, Hernandez and Brown",Brand_15,"Anderson, Hernandez And Brown"
8096,Brand_5,Product_162,98.95,Medium,2920,A,22.13,V017,"Anderson, Hernandez and Brown",Brand_5,"Anderson, Hernandez And Brown"
2767,Brand_11,Product_3,278.20,Small,9039,B,86.39,V017,"Anderson, Hernandez and Brown",Brand_11,"Anderson, Hernandez And Brown"
13881,Brand_7,Product_40,258.67,Medium,11173,A,259.18,V017,"Anderson, Hernandez and Brown",Brand_7,"Anderson, Hernandez And Brown"
...,...,...,...,...,...,...,...,...,...,...,...
1537,Brand_20,Product_1,225.13,Large,6698,B,94.52,V017,Wright-Morrison,Brand_20,Wright Morrison
18662,Brand_8,Product_114,102.77,Large,3191,C,226.37,V017,Wright-Morrison,Brand_8,Wright Morrison
7351,Brand_7,Product_74,106.01,Large,13404,B,123.52,V017,Wright-Morrison,Brand_7,Wright Morrison
13824,Brand_2,Product_53,173.68,Medium,311,A,42.84,V017,Wright-Morrison,Brand_2,Wright Morrison


In [81]:
wb2 = dfs['vendor_invoice'][dfs['vendor_invoice']['VendorNumber'] == 'V017'].sort_values('VendorName_clean')
wb2['VendorName'].nunique()
wb2

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval,VendorName_clean,PayDate_clean
1632,V017,"Anderson, Hernandez and Brown",04/17/2024,PO-1296,11/07/2024,02/04/2024,899,54095.64,1862.40,Pending,"Anderson, Hernandez And Brown",2024-10-04
2494,V017,"Anderson, Hernandez and Brown","Jan 27, 2024",PO-0858,01/28/2024,2024-01-19,719,12183.73,1829.41,Pending,"Anderson, Hernandez And Brown",2024-09-04
4609,V017,"Anderson, Hernandez and Brown",04/20/2024,PO-1871,01/25/2024,11/21/2024,78,3797.98,3682.16,Pending,"Anderson, Hernandez And Brown",2024-03-09
4472,V017,"Anderson, Hernandez and Brown",2024-01-16,PO-0099,07/31/2024,"Nov 08, 2024",1376,53881.31,2521.67,Rejected,"Anderson, Hernandez And Brown",2024-10-26
2521,V017,"Brown, Fitzgerald and Evans",2024-04-04,PO-0169,01/10/24,08/01/24,1288,35630.84,3903.98,Approved,"Brown, Fitzgerald And Evans",2024-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...
1176,V017,"Williams,_Hickman_and_Nichols","Mar 12, 2024",PO-1873,09/04/2024,24/06/24,1973,24649.19,3863.60,Approved,"Williams, Hickman And Nichols",2024-12-01
1752,V017,"Williams,_Hickman_and_Nichols","Nov 14, 2024",PO-6223,"Jul 25, 2024",2024-12-12,1512,28167.94,690.18,Approved,"Williams, Hickman And Nichols",2024-07-12
2558,V017,"Williams,_Hickman_and_Nichols",2024-07-09,PO-8478,2024-08-31,08/28/2024,124,99953.43,3083.21,Approved,"Williams, Hickman And Nichols",2024-09-05
2169,V017,WRIGHT-MORRISON,2024-07-05,PO-7102,"Apr 29, 2024",22/03/24,1400,34106.92,1228.89,Approved,Wright Morrison,2024-01-09


In [84]:
set(wb1.columns) & set(wb2.columns)

{'VendorName', 'VendorName_clean', 'VendorNumber'}